# Munge Miriam Summary

Miriam has created a summary table with the protein trap image scores. There is some munging that needs to happen before merging onto the big table.

In [3]:
import os
import sys
from pathlib import Path
from io import StringIO

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
from larval_gonad.notebook import Nb

# Setup notebook
nbconfig = Nb.setup_notebook()

last updated: 2018-07-25 
Git hash: ec590ad53c8c96d2ae0d3ae6d39670b1f8b4c189


In [20]:
# This data is from a spreadsheet that Miriam gave me. I had to add on FBgns
dat = """
FBgn,gene_symbol,GSCs/Gonia,Early 1º,Mid 1º,Late 1º,CySCs/Cytes,Pigment,Terminal Epithelium,Hub
FBgn0026573,ADD1,1.25,3.75,6.25,6.875,0,0,0,0
FBgn0012037,Ance,1.25,1.875,3.333333333,3.75,4.166666667,0,0,0
FBgn0011206,bol,0,0.625,2.5,6.875,0,0,0,0
FBgn0015609,CadN,1.875,1.875,4.166666667,5,3.125,1.666666667,1.875,2.5
FBgn0015609,CadN,1.875,1.875,2.5,1.666666667,3.125,0.625,0.625,2.5
FBgn0015609,CadN,0.833333333,0.833333333,1.666666667,1.25,2.5,1.25,1.25,1.25
FBgn0032771,CG17349,1.666666667,2.5,2.5,2.5,2.5,0,0,2.5
FBgn0264494,CG17646,0.625,1.875,1.875,2.5,2.5,0,0.625,0
FBgn0031518,CG3277,1.25,1.25,1.666666667,1.666666667,2.5,0.833333333,1.25,1.875
FBgn0259221,CG42321,1.25,1.666666667,1.666666667,1.666666667,2.5,0.625,0,1.25
FBgn0039754,CG9747,0.833333333,2.5,3.125,2.5,1.875,1.25,1.25,1.25
FBgn0027598,cindr,1.666666667,2.5,2.5,2.5,3.333333333,3.75,4.166666667,3.333333333
FBgn0026533,Dek,7.5,7.5,7.5,7.5,3.75,7.5,3.75,7.5
FBgn0087008,e(y)3,6.25,6.25,6.25,5,3.125,5.833333333,3.75,6.25
FBgn0051158,Efa6,2.5,3.333333333,3.333333333,3.333333333,5,,5.833333333,
FBgn0051158,Efa6,2.5,4.375,4.375,4.375,2.5,1.25,1.25,3.125
FBgn0000636,Fas3,0.625,0.625,1.25,1.25,1.25,0,5.833333333,7.5
FBgn0000636,Fas3,0,1.25,0,1.25,1.25,1.25,4.375,5.625
FBgn0013469,klu,1.666666667,3.333333333,3.333333333,3.333333333,4.166666667,2.5,1.666666667,1.25
FBgn0034282,Mapmodulin,5.833333333,5.833333333,6.666666667,4.166666667,4.166666667,4.166666667,5,2.5
FBgn0265487,mbl,0.625,2.5,6.25,4.375,1.666666667,3.75,6.25,0.625
FBgn0262519,Mi-2,0,1.25,1.25,1.25,2.5,0,0,0
FBgn0030505,NFAT,0.833333333,0.833333333,1.666666667,1.875,2.5,4.166666667,0,0.625
FBgn0002945,nkd,1.25,2.5,3.333333333,1.666666667,1.875,0.833333333,1.25,1.666666667
FBgn0050418,nord,0.625,1.875,1.875,1.875,3.75,0.625,7.5,0
FBgn0264975,Nrg,3.75,1.875,1.875,1.875,5.625,3.333333333,6.875,6.25
FBgn0261885,osa,6.25,5.625,5.625,4.375,3.125,6.666666667,6.25,4.375
FBgn0039044,p53,4.375,3.125,1.875,0,1.875,0.625,0.625,0.625
FBgn0030520,Pdcd4,7.5,6.875,6.25,5.625,6.875,2.5,7.5,3.75
FBgn0085370,Pde11,0,0,0,0,0.625,0,0,0
FBgn0264953,Piezo,1.875,0,0,1.25,3.333333333,2.5,5.625,1.875
FBgn0000416,Sap-r,3.125,3.75,3.75,5,5.833333333,2.5,6.875,0
FBgn0003475,spir,0,0.625,1.25,1.25,0.833333333,0,0,0
FBgn0026370,SRPK,5.625,6.25,6.875,6.875,3.125,0.625,0.625,0.833333333
FBgn0041182,Tep2,1.25,1.25,1.875,2.5,4.375,6.25,1.25,0.833333333
FBgn0004885,tok,0.625,1.666666667,1.875,1.875,0.833333333,0,1.875,0.625
FBgn0010473,tutl,0,0.625,0.625,0.625,0.625,0,1.25,0
FBgn0011725,twin,0.833333333,0.833333333,0.833333333,0.833333333,0,0,0,0
"""

df = pd.read_csv(StringIO(dat), sep=',')

agg = df.groupby(['FBgn', 'gene_symbol']).mean()

In [21]:
agg

,,GSCs/Gonia,Early 1º,Mid 1º,Late 1º,CySCs/Cytes,Pigment,Terminal Epithelium,Hub
FBgn,gene_symbol,,,,,,,,
FBgn0000416,Sap-r,3.125000,3.750000,3.750000,5.000000,5.833333,2.500000,6.875000,0.000000
FBgn0000636,Fas3,0.312500,0.937500,0.625000,1.250000,1.250000,0.625000,5.104167,6.562500
FBgn0002945,nkd,1.250000,2.500000,3.333333,1.666667,1.875000,0.833333,1.250000,1.666667
FBgn0003475,spir,0.000000,0.625000,1.250000,1.250000,0.833333,0.000000,0.000000,0.000000
FBgn0004885,tok,0.625000,1.666667,1.875000,1.875000,0.833333,0.000000,1.875000,0.625000
FBgn0010473,tutl,0.000000,0.625000,0.625000,0.625000,0.625000,0.000000,1.250000,0.000000
FBgn0011206,bol,0.000000,0.625000,2.500000,6.875000,0.000000,0.000000,0.000000,0.000000
FBgn0011725,twin,0.833333,0.833333,0.833333,0.833333,0.000000,0.000000,0.000000,0.000000
FBgn0012037,Ance,1.250000,1.875000,3.333333,3.750000,4.166667,0.000000,0.000000,0.000000


In [22]:
agg.to_csv('../data/external/2018-07-27_munge_miriam_.tsv', sep='\t')